In [ ]:
import konlpy

In [ ]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import re
import tensorflow as tf
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 형태소 분석


In [ ]:
dic = pd.read_csv('result_file.csv',encoding = 'UTF-8-sig')
comment_data = dic[['comment']] # 선플 cvs 파일에서 판다만 

In [ ]:
#한글과 공백을 제외하고 모두 제거
normalized_text = []
for string in comment_data.tolist():
    try:
        tokens = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣]+", " ", "?", "!", "~", ".", string.lower()) # 문장을 만들 때 문장부호를 사용할 수 있도록 추가해서 학습시킴
    except Exception as e:
        print(string)
        break
    normalized_text.append(tokens)
print(normalized_text[:5])

In [ ]:
comment_data.comment = normalized_text
for sentence in comment_data['comment']:
    print(sentence)

    # commnet_data 자료구조 어떻게 되어있나 살펴보고 이 부분 refactoring 하기

In [ ]:
# 토큰화
kmr = Komoran("EXP")
X_token=[]
for sentence in comment_data['comment']:
    temp_X = []
    temp_X = kmr.morphes(sentence)
    #temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_token.append(temp_X)

# 정수 인코딩 수행

In [ ]:
max_words = 20000
tokenizer = Tokenizer(num_words = max_words) # 상위 20,000개의 단어만 보존
tokenizer.fit_on_texts(X_token) 
X_token = tokenizer.texts_to_sequences(X_token)

In [ ]:
word_to_index = tokenizer.word_index

In [ ]:
vocab_size = len(word_to_index)+1
print('형태소 집합의 크기: {}'.format((vocab_size)))
max_len = max(len(w)) for w in X_token)
print('형태소의 최대 길이 : {}'.format(max_len))

In [ ]:
X_data = pad_sequences(X_token, maxlen=max_len)
print("data shape: ", X_data.shape)

In [ ]:
y_data = np.array(y_data).reshape(-1, 1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size= 0.3, random_state=1234)

# RNN

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential

In [ ]:
from tensorflow.keras.layers import SimpleRNN

model = Sequential()
model.add(Embedding(vocab_size, 32)) # 임베딩 벡터의 차원은 32
model.add(SimpleRNN(32)) # RNN 셀의 hidden_size는 32
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# 문장 생성

In [ ]:
def sentence_generation(model, t, current_word): # 모델, 토크나이저, 현재 단어
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    while(1):
        encoded = t.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5)
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
        '''if word가 종결어미or!~.?^이면, break'''
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [ ]:
print(sentence_generation(model, t, current_word))